In [ ]:
import sys
sys.path.append('../src/utils')

from ac_object_etl_functions import *
#%load_ext memray
# Load in the data labels
params = {}
#params['category_index_column'] = "present1hot"
params['category_labels'] = ['No', 'Yes']
params['images_folder'] = '../data/images/ac_150_images'
params["data_labels_path"] = '../data/labels/ac_150_labels_jess_notes.csv'
params["annotation_path"] = '../data/labels/Export v2 project - AC Unit Evaluation Test - 7_3_2024.ndjson'
params["iou_threshold"] = 1.0/3
params["annotators_per_image"] = 5
# params['kernal'] = np.ones((12,12), np.uint8)
# params['hsv_range'] = [[8, 120, 200], [12, 255, 255]]
# params['min_ratio'] = 2
params['dataset_version'] = 'v4'
params['rerun_ds_generation'] = True

prime_labels = load_ac_data_spreadsheet(params)
prime_labels = prime_labels.rename(columns = {'row_number ': 'row_number'})
label_set = pd.read_json(params["annotation_path"], lines=True)
df = extract_label_data(label_set)
#join prime_labels row_number to df with image_id
df = pd.merge(df, prime_labels[['image_id','row_number']], on = 'image_id', how = 'inner')
df = df.rename(columns = {'row_number': 'image_orig_idx'})

#remove intersecting bounding box (specific to ../data/images/ac_150)
df = df[df['feature_id'] != 'clxgcq78i00hx3b6qit987ryb']

matched_sets = generate_matched_sets(df, params["iou_threshold"], params["annotators_per_image"])

In [ ]:
#May be greater than 100 you have intersecting sets, need to resolve with filter
agree_bin = matched_sets.groupby(['name','percent_agreement']).size().rename('count').reset_index().join(matched_sets.groupby(['name']).size().rename('total'), on = 'name')
agree_bin['percent_total'] = agree_bin['count']/agree_bin['total']*100
print(agree_bin)


In [ ]:
filt_connected_sets = matched_sets[matched_sets['num_agree_in_set'] >= 3]


In [ ]:
min_matches_per_image = 3

worker_has_polygon = df[['image_id', 'name', 'worker']].drop_duplicates()
worker_has_polygon['ind_worker_label'] = int(1)
worker_has_polygon['cross_join'] = 1

#labeler_df = connected_sets.copy(deep = True)
labeler_df = matched_sets[matched_sets['num_agree_in_set'] >= min_matches_per_image]
labeler_df = labeler_df[['image_id','name']].drop_duplicates()
comp_to_prime = labeler_df.reset_index(drop=False)
comp_to_prime['labeler_consensus'] = 1
ac_comp = comp_to_prime[comp_to_prime['name'] == 'AC Unit']
ac_comp = pd.merge(ac_comp, prime_labels[['image_id','present1hot']], on = ['image_id'], how = 'outer').rename(columns = {'present1hot': 'prime_label'})
ac_comp['name'] = 'AC Unit'
#ac_comp['averaged_polygons'] = ac_comp['averaged_polygons'].fillna(0)
leak_comp = comp_to_prime[comp_to_prime['name'] == 'AC leaking']
leak_comp = pd.merge(leak_comp, prime_labels[['image_id','leaking1hot']], on = ['image_id'], how = 'outer').rename(columns = {'leaking1hot': 'prime_label'})
leak_comp['name'] = 'AC leaking'
#leak_comp['averaged_polygons'] = leak_comp['averaged_polygons'].fillna(0)
comp_to_prime = pd.concat([ac_comp,leak_comp],axis=0,ignore_index=True)
comp_to_prime['labeler_consensus'] = comp_to_prime['labeler_consensus'].fillna(0)

comp_to_prime['match'] = (comp_to_prime['prime_label'] == comp_to_prime['labeler_consensus']).astype(int)

comp_to_prime['hit'] = ((comp_to_prime['prime_label'] == 1) & (comp_to_prime['labeler_consensus'] == 1)).astype(int)
comp_to_prime['correct_rejection'] = ((comp_to_prime['prime_label'] == 0) & (comp_to_prime['labeler_consensus'] == 0)).astype(int)
comp_to_prime['false_alarm'] = ((comp_to_prime['prime_label'] == 0) & (comp_to_prime['labeler_consensus'] == 1)).astype(int)
comp_to_prime['miss'] = ((comp_to_prime['prime_label'] == 1) & (comp_to_prime['labeler_consensus'] == 0)).astype(int)
comp_to_prime['cross_join'] = 1

worker_comp_to_prime = pd.merge(comp_to_prime, worker_has_polygon[['worker','cross_join']].drop_duplicates(), on = ['cross_join'], how = 'outer').fillna(0)
comp_to_prime = comp_to_prime.drop(columns = ['cross_join']).fillna(0)
worker_comp_to_prime = worker_comp_to_prime.drop(columns = ['cross_join'])
worker_has_polygon = worker_has_polygon.drop(columns = ['cross_join'])

worker_comp_to_prime = pd.merge(worker_comp_to_prime, worker_has_polygon, on = ['image_id','name','worker'], how = 'outer').fillna(0)

In [ ]:
confusion_matrix = pd.crosstab(comp_to_prime.loc[(comp_to_prime['name'] == 'AC Unit'),'prime_label'], comp_to_prime['labeler_consensus'], rownames=['True'], colnames=['Predicted'], margins=True)

print(confusion_matrix)

In [ ]:
confusion_matrix = pd.crosstab(comp_to_prime.loc[(comp_to_prime['name'] == 'AC leaking'),'prime_label'], comp_to_prime['labeler_consensus'], rownames=['True'], colnames=['Predicted'], margins=True)

print(confusion_matrix)

In [ ]:
# for worker in worker_comp_to_prime['worker'].unique(): 
#     comp_filt = ((worker_comp_to_prime['name'] == 'AC Unit') & (worker_comp_to_prime['worker'] == worker))
#     confusion_matrix = pd.crosstab(worker_comp_to_prime.loc[comp_filt,'prime_label'], worker_comp_to_prime.loc[(worker_comp_to_prime['worker'] == worker),'ind_worker_label'], rownames=['True'], colnames=['Predicted'], margins=True)
# 
#     print(worker + ': ')
#     print(confusion_matrix)
#     print(' ')

In [ ]:
# for worker in worker_comp_to_prime['worker'].unique(): 
#     comp_filt = ((worker_comp_to_prime['name'] == 'AC leaking') & (worker_comp_to_prime['worker'] == worker))
#     confusion_matrix = pd.crosstab(worker_comp_to_prime.loc[comp_filt,'prime_label'], worker_comp_to_prime.loc[(worker_comp_to_prime['worker'] == worker),'ind_worker_label'], rownames=['True'], colnames=['Predicted'], margins=True)
# 
#     print(worker + ': ')
#     print(confusion_matrix)
#     print(' ')

In [ ]:
def calculate_f1_score(confusion_matrix):
    # Calculate precision and recall from the confusion matrix
    TruePositives = confusion_matrix[1][1]
    FalsePositives = confusion_matrix[0][1]
    FalseNegatives = confusion_matrix[1][0]

    precision = TruePositives / (TruePositives + FalsePositives)
    recall = TruePositives / (TruePositives + FalseNegatives)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall)

    return f1_score

confusion_matrix = pd.crosstab(comp_to_prime.loc[(comp_to_prime['name'] == 'AC Unit'),'prime_label'], comp_to_prime['labeler_consensus'], rownames=['True'], colnames=['Predicted'], margins=True)
f1_score_leaking = calculate_f1_score(confusion_matrix)
print(confusion_matrix)
print('AC Unit: ' + str(f1_score_leaking))

# Assuming 'confusion_matrix' is the confusion matrix you have calculated
confusion_matrix = pd.crosstab(comp_to_prime.loc[(comp_to_prime['name'] == 'AC leaking'),'prime_label'], comp_to_prime['labeler_consensus'], rownames=['True'], colnames=['Predicted'], margins=True)
f1_score_leaking = calculate_f1_score(confusion_matrix)
print(confusion_matrix)
print('AC leaking: ' + str(f1_score_leaking))



In [ ]:
worker_has_polygon = df[['image_id', 'name', 'worker']].drop_duplicates()
worker_has_polygon['ind_worker_label'] = int(1)
worker_has_polygon['cross_join'] = 1

labeler_df = matched_sets.copy(deep = True)
comp_to_prime = labeler_df.reset_index(drop=False)
comp_to_prime['labeler_consensus'] = 1
ac_comp = comp_to_prime[comp_to_prime['name'] == 'AC Unit']
ac_comp = pd.merge(ac_comp, prime_labels[['image_id','present1hot']], on = ['image_id'], how = 'outer').rename(columns = {'present1hot': 'prime_label'}).fillna(0)
ac_comp['name'] = 'AC Unit'
#ac_comp['averaged_polygons'] = ac_comp['averaged_polygons'].fillna(0)
leak_comp = comp_to_prime[comp_to_prime['name'] == 'AC leaking']
leak_comp = pd.merge(leak_comp, prime_labels[['image_id','leaking1hot']], on = ['image_id'], how = 'outer').rename(columns = {'leaking1hot': 'prime_label'}).fillna(0)
leak_comp['name'] = 'AC leaking'
#leak_comp['averaged_polygons'] = leak_comp['averaged_polygons'].fillna(0)
comp_to_prime = pd.concat([ac_comp,leak_comp],axis=0,ignore_index=True)
comp_to_prime['labeler_consensus'] = comp_to_prime['labeler_consensus'].fillna(0)

comp_to_prime['match'] = (comp_to_prime['prime_label'] == comp_to_prime['labeler_consensus']).astype(int)

comp_to_prime['hit'] = ((comp_to_prime['prime_label'] == 1) & (comp_to_prime['labeler_consensus'] == 1)).astype(int)
comp_to_prime['correct_rejection'] = ((comp_to_prime['prime_label'] == 0) & (comp_to_prime['labeler_consensus'] == 0)).astype(int)
comp_to_prime['false_alarm'] = ((comp_to_prime['prime_label'] == 0) & (comp_to_prime['labeler_consensus'] == 1)).astype(int)
comp_to_prime['miss'] = ((comp_to_prime['prime_label'] == 1) & (comp_to_prime['labeler_consensus'] == 0)).astype(int)
comp_to_prime['cross_join'] = 1

In [ ]:
for agreement in comp_to_prime['percent_agreement'].unique(): 
    comp_filt = ((comp_to_prime['name'] == 'AC Unit') & (comp_to_prime['percent_agreement'] == agreement))
    confusion_matrix = pd.crosstab(comp_to_prime.loc[comp_filt,'prime_label'], comp_to_prime.loc[(comp_to_prime['percent_agreement'] == agreement),'labeler_consensus'], rownames=['True'], colnames=['Predicted'], margins=True)

    print(str(agreement) + ': ')
    print(confusion_matrix)
    print(' ')

In [ ]:
accuracy_by_agreement = comp_to_prime.groupby(['name','percent_agreement'])['match'].mean().rename('percent_accuracy')*100

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'accuracy_by_agreement' is your DataFrame
accuracy_by_agreement = accuracy_by_agreement.reset_index()

# Create a pivot table
pivot_table = accuracy_by_agreement.pivot(index='percent_agreement', columns='name', values='percent_accuracy')

# Plot the data
pivot_table.plot(kind='bar', stacked=False)

# Add labels and title
plt.xlabel('% Labelers Match on Object Detection')
plt.ylabel('% Accuracy')
plt.title('Accuracy by % Labelers Match on Object Detection')
legend = plt.legend(title='Object Type', loc='lower left')
# Show the plot
plt.show()